In [14]:
"""
实验目的：实现基于CNN、RNN的文本分类

实验内容：
1）词嵌入初始化方式：随机embedding、加载glove
2）CNN/RNN的特征抽取
3）Dropout


参考：
https://arxiv.org/abs/1408.5882
https://github.com/yokusama/CNN_Sentence_Classification
https://torchtext.readthedocs.io/en/latest/
http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/bidirectional_recurrent_neural_network/main.py#L39-L58

"""

'\n实验目的：实现基于CNN、RNN的文本分类\n\n实验内容：\n1）词嵌入初始化方式：随机embedding、加载glove\n2）CNN/RNN的特征抽取\n3）Dropout\n\n\n参考：\nhttps://arxiv.org/abs/1408.5882\nhttps://github.com/yokusama/CNN_Sentence_Classification\nhttps://torchtext.readthedocs.io/en/latest/\nhttp://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/\nhttps://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/\nhttps://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/bidirectional_recurrent_neural_network/main.py#L39-L58\n\n'

In [15]:
import os
import time
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sklearn

print(os.getcwd())


dir_all_data = "data\\task2_all_data.tsv"

# 超参数设置
BATCH_SIZE = 10
cpu = True  # True   False
if cpu:
    USE_CUDA = False
    DEVICE = torch.device("cpu")
else:
    USE_CUDA = torch.cuda.is_available()
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.cuda.set_device(0)

f:\W_project\ML_and_DL\nlp-beginner_solution\Task2


In [16]:
import torch

print(torch.__version__)

2.5.1+cu124


In [17]:
# 从文件中读取数据
data_all = pd.read_csv(dir_all_data, sep="\t")
# print(all_data.shape)    #(156060, 4)
# print(all_data.keys())   #['PhraseId', 'SentenceId', 'Phrase', 'Sentiment']
idx = np.arange(data_all.shape[0])
# print(data_all.head())
# print(type(idx))   #<class 'numpy.ndarray'>

In [18]:
# shuffle、划分验证集、测试集,并保存
seed = 0
np.random.seed(seed)
# print(idx)
np.random.shuffle(idx)
# print(idx)

train_size = int(len(idx) * 0.6)
test_size = int(len(idx) * 0.8)

data_all.iloc[idx[:train_size], :].to_csv("data/task2_train.csv", index=False)
data_all.iloc[idx[train_size:test_size], :].to_csv("data/task2_test.csv", index=False)
data_all.iloc[idx[test_size:], :].to_csv("data/task2_dev.csv", index=False)

In [19]:
# 使用Torchtext采用声明式方法加载数据
from torchtext import data

PAD_TOKEN = "<pad>"
TEXT = data.Field(sequential=True, batch_first=True, lower=True, pad_token=PAD_TOKEN)
LABEL = data.Field(sequential=False, batch_first=True, unk_token=None)

In [22]:
# 读取数据
datafields = [
    ("PhraseId", None),  # 不需要的filed设置为None
    ("SentenceId", None),
    ("Phrase", TEXT),
    ("Sentiment", LABEL),
]
train_data = data.TabularDataset(
    path="data/task2_train.csv", format="csv", fields=datafields
)
dev_data = data.TabularDataset(
    path="data/task2_dev.csv", format="csv", fields=datafields
)
test_data = data.TabularDataset(
    path="data/task2_test.csv", format="csv", fields=datafields
)
train_data

In [31]:
# 构建词典，字符映射到embedding
# TEXT.vocab.vectors 就是词向量
from torchtext.vocab import Vectors

glove_vectors = Vectors(
    "F:\\W_project\\ML_and_DL\\nlp-beginner_solution\\txt_classify\\glove.6B\\glove.6B.50d.txt",
    cache="nlp-beginner_solution\\txt_classify",
)
TEXT.build_vocab(
    train_data,
    vectors=glove_vectors,
    unk_init=lambda x: torch.nn.init.uniform_(x, a=-0.25, b=0.25),
)
LABEL.build_vocab(train_data)


# 得到索引，PAD_TOKEN='<pad>'
PAD_INDEX = TEXT.vocab.stoi[PAD_TOKEN]
TEXT.vocab.vectors[PAD_INDEX] = 0.0

100%|█████████▉| 400000/400001 [00:14<00:00, 27254.44it/s]


In [35]:
TEXT

In [32]:
# 构建迭代器
train_iterator = data.BucketIterator(
    train_data, batch_size=BATCH_SIZE, train=True, shuffle=True, device=DEVICE
)

dev_iterator = data.Iterator(
    dev_data, batch_size=len(dev_data), train=False, sort=False, device=DEVICE
)

test_iterator = data.Iterator(
    test_data, batch_size=len(test_data), train=False, sort=False, device=DEVICE
)

In [33]:
# 部分参数设置
embedding_choice = "glove"  #  'static'    'non-static'
num_embeddings = len(TEXT.vocab)
embedding_dim = 50
dropout_p = 0.5
filters_num = 100

vocab_size = len(TEXT.vocab)
label_num = len(LABEL.vocab)
print(vocab_size, label_num)

16473 6


In [36]:
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.embedding_choice = embedding_choice

        if self.embedding_choice == "rand":
            self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        if self.embedding_choice == "glove":
            self.embedding = nn.Embedding(
                num_embeddings, embedding_dim, padding_idx=PAD_INDEX
            ).from_pretrained(TEXT.vocab.vectors, freeze=True)

        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=filters_num,  # 卷积产生的通道
            kernel_size=(3, embedding_dim),
            padding=(2, 0),
        )

        self.conv2 = nn.Conv2d(
            in_channels=1,
            out_channels=filters_num,  # 卷积产生的通道
            kernel_size=(4, embedding_dim),
            padding=(3, 0),
        )

        self.conv3 = nn.Conv2d(
            in_channels=1,
            out_channels=filters_num,  # 卷积产生的通道
            kernel_size=(5, embedding_dim),
            padding=(4, 0),
        )

        self.dropout = nn.Dropout(dropout_p)

        self.fc = nn.Linear(filters_num * 3, label_num)

    def forward(self, x):  # (Batch_size, Length)
        x = self.embedding(x).unsqueeze(1)  # (Batch_size, Length, Dimention)
        # (Batch_size, 1, Length, Dimention)

        x1 = F.relu(self.conv1(x)).squeeze(
            3
        )  # (Batch_size, filters_num, length+padding, 1)
        # (Batch_size, filters_num, length+padding)
        x1 = F.max_pool1d(x1, x1.size(2)).squeeze(2)  # (Batch_size, filters_num, 1)
        # (Batch_size, filters_num)

        x2 = F.relu(self.conv2(x)).squeeze(3)
        x2 = F.max_pool1d(x2, x2.size(2)).squeeze(2)

        x3 = F.relu(self.conv3(x)).squeeze(3)
        x3 = F.max_pool1d(x3, x3.size(2)).squeeze(2)

        x = torch.cat((x1, x2, x3), dim=1)  # (Batch_size, filters_num *3 )
        x = self.dropout(x)  # (Batch_size, filters_num *3 )
        out = self.fc(x)  # (Batch_size, label_num  )
        return out

In [37]:
# 构建模型

model = CNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 创建优化器SGD
criterion = nn.CrossEntropyLoss()  # 损失函数

if USE_CUDA:
    model.cuda()

In [38]:
# 开始训练
import time

epoch = 100
best_accuracy = 0.0
start_time = time.time()

for i in range(epoch):
    model.train()
    total_loss = 0.0
    accuracy = 0.0
    total_correct = 0.0
    total_data_num = len(train_iterator.dataset)
    steps = 0.0
    # 训练
    for batch in train_iterator:
        steps += 1
        # print(steps)
        optimizer.zero_grad()  #  梯度缓存清零

        batch_text = batch.Phrase
        batch_label = batch.Sentiment
        out = model(batch_text)  # [batch_size, label_num]
        loss = criterion(out, batch_label)
        total_loss = total_loss + loss.item()

        loss.backward()
        optimizer.step()

        correct = (
            torch.max(out, dim=1)[1].view(batch_label.size())  # get the indices
            == batch_label
        ).sum()
        total_correct = total_correct + correct.item()

        if steps % 100 == 0:
            print(
                "Epoch %d_%.3f%%:  Training average Loss: %f"
                % (
                    i,
                    steps
                    * train_iterator.batch_size
                    * 100
                    / len(train_iterator.dataset),
                    total_loss / steps,
                )
            )

    # 每个epoch都验证一下
    model.eval()
    total_loss = 0.0
    accuracy = 0.0
    total_correct = 0.0
    total_data_num = len(dev_iterator.dataset)
    steps = 0.0
    for batch in dev_iterator:
        steps += 1
        batch_text = batch.Phrase
        batch_label = batch.Sentiment
        out = model(batch_text)
        loss = criterion(out, batch_label)
        total_loss = total_loss + loss.item()

        correct = (
            torch.max(out, dim=1)[1].view(batch_label.size()) == batch_label
        ).sum()
        total_correct = total_correct + correct.item()

        print(
            "Epoch %d :  Verification average Loss: %f, Verification accuracy: %f%%,Total Time:%f"
            % (
                i,
                total_loss / steps,
                total_correct * 100 / total_data_num,
                time.time() - start_time,
            )
        )

        if best_accuracy < total_correct / total_data_num:
            best_accuracy = total_correct / total_data_num
            torch.save(
                model,
                "model_dict/model_glove/epoch_%d_accuracy_%f"
                % (i, total_correct / total_data_num),
            )
            print(
                "Model is saved in model_dict/model_glove/epoch_%d_accuracy_%f"
                % (i, total_correct / total_data_num)
            )
            # torch.cuda.empty_cache()
    break  # 运行时去除break

Epoch 0_1.068%:  Training average Loss: 1.370474
Epoch 0_2.136%:  Training average Loss: 1.336910
Epoch 0_3.204%:  Training average Loss: 1.293310
Epoch 0_4.272%:  Training average Loss: 1.266158
Epoch 0_5.340%:  Training average Loss: 1.234342
Epoch 0_6.408%:  Training average Loss: 1.213062
Epoch 0_7.476%:  Training average Loss: 1.195165
Epoch 0_8.544%:  Training average Loss: 1.182784
Epoch 0_9.612%:  Training average Loss: 1.174780
Epoch 0_10.680%:  Training average Loss: 1.166213
Epoch 0_11.747%:  Training average Loss: 1.162433
Epoch 0_12.815%:  Training average Loss: 1.154645
Epoch 0_13.883%:  Training average Loss: 1.148871
Epoch 0_14.951%:  Training average Loss: 1.144656
Epoch 0_16.019%:  Training average Loss: 1.142490
Epoch 0_17.087%:  Training average Loss: 1.139114
Epoch 0_18.155%:  Training average Loss: 1.135034
Epoch 0_19.223%:  Training average Loss: 1.132984
Epoch 0_20.291%:  Training average Loss: 1.128309
Epoch 0_21.359%:  Training average Loss: 1.124403
Epoch 0_2

RuntimeError: Parent directory model_dict/model_glove does not exist.

In [ ]:
# 测试-重新读取文件（方便重写成.py文件）
PATH = "model_dict/model_glove/epoch_0_accuracy_0.586647"
model = torch.load(PATH)

total_loss = 0.0
accuracy = 0.0
total_correct = 0.0
total_data_num = len(train_iterator.dataset)
steps = 0.0
start_time = time.time()
for batch in test_iterator:
    steps += 1
    batch_text = batch.Phrase
    batch_label = batch.Sentiment
    out = model(batch_text)
    loss = criterion(out, batch_label)
    total_loss = total_loss + loss.item()

    correct = (torch.max(out, dim=1)[1].view(batch_label.size()) == batch_label).sum()
    total_correct = total_correct + correct.item()
    # break

print(
    "Test average Loss: %f, Test accuracy: %f，Total time: %f"
    % (total_loss / steps, total_correct / total_data_num, time.time() - start_time)
)